In [4]:
# Import required packages
import gc
import os
import shutil

import keras.callbacks as kc
import keras.optimizers as ko
import keras.utils as ku
import numpy as np
import sklearn.utils as sku
import tensorflow as tf

import Config as conf
import CSV as csv
from ResNet import ResnetBuilder, bottleneck

In [2]:
# Import & shuffle CSV data
xx, yy = csv.getCSV()
xx, yy = sku.shuffle(xx, yy, random_state=0)
xx = xx[..., np.newaxis]

Importing CSV files...
 -- Loading: ./Input_WINDOW3760_COL90_TH95/csi_001.csv
 -- Loading: ./Input_WINDOW3760_COL90_TH95/csi_002.csv
 -- Loading: ./Input_WINDOW3760_COL90_TH95/csi_003.csv
 -- Loading: ./Input_WINDOW3760_COL90_TH95/csi_004.csv
 -- Loading: ./Input_WINDOW3760_COL90_TH95/csi_005.csv
 -- Loading: ./Input_WINDOW3760_COL90_TH95/csi_006.csv
 -- Loading: ./Input_WINDOW3760_COL90_TH95/action_001.csv
 -- Loading: ./Input_WINDOW3760_COL90_TH95/action_002.csv
 -- Loading: ./Input_WINDOW3760_COL90_TH95/action_003.csv
 -- Loading: ./Input_WINDOW3760_COL90_TH95/action_004.csv
 -- Loading: ./Input_WINDOW3760_COL90_TH95/action_005.csv
 -- Loading: ./Input_WINDOW3760_COL90_TH95/action_006.csv
 -- Reshaping xx ...
 -- Shape notice: [xx] (1200, 3760, 90) [yy] (1200, 2)
 -- Using No-Activity window.
Loaded CSVs


In [5]:
# Setup Keras DenseNet Model (DenseNet-BC)
conf.N_FILTERS = 20  # 40
conf.DEPTH = 34  # 40
nadam = ko.Nadam(lr=conf.LEARNING_RATE)
adam = ko.Adam(lr=conf.LEARNING_RATE, amsgrad=True)
model = ResnetBuilder.build(
    (1, conf.WINDOW_SIZE, conf.N_COLUMNS),
    conf.USE_NOACTIVITY and conf.N_CLASSES or conf.N_VALID_CLASSES,
    bottleneck,[3,4,6,3]
)
model.compile(
    loss="categorical_crossentropy",
    optimizer=nadam,
    metrics=["accuracy"]
)
print("GC Collected:", gc.collect())

GC Collected: 8


In [6]:
# Check output directory and prepare tensorboard
outputDir = conf.OUTPUT_PATH.format(conf.LEARNING_RATE, conf.BATCH_SIZE,
                                    conf.N_FILTERS, "")
if os.path.exists(outputDir):
    shutil.rmtree(outputDir)
os.makedirs(outputDir)
logDir = conf.LOG_PATH.format(conf.LEARNING_RATE, conf.BATCH_SIZE,
                              conf.N_FILTERS, "")
if os.path.exists(logDir):
    shutil.rmtree(logDir)
os.makedirs(logDir)
tensorboard = kc.TensorBoard(
    log_dir=logDir,
    histogram_freq=0,
    batch_size=conf.BATCH_SIZE,
    write_graph=True,
    write_grads=True,
    write_images=True,
    update_freq=10)
print(
    "If you have tensorboard in this environment, you can type below to see the result in tensorboard:"
)
print("    tensorboard --logdir=" + logDir)
print("Keras checkpoints and final result will be saved in here:")
print("    " + outputDir)

If you have tensorboard in this environment, you can type below to see the result in tensorboard:
    tensorboard --logdir=./Log_LR0.002_BATCH64_GWRRATE20/
Keras checkpoints and final result will be saved in here:
    ./Output_LR0.002_BATCH64_GWRRATE20/


In [ ]:
# Run KFold
xxdnc = {}
yydnc = {}
for i in range(conf.DNC):
    xx = np.roll(xx, int(len(xx) / conf.DNC), axis=0)
    yy = np.roll(yy, int(len(yy) / conf.DNC), axis=0)
    xxdnc[i] = xx[:int(len(xx) / conf.DNC)]
    yydnc[i] = yy[:int(len(yy) / conf.DNC)]
    
for n in range(conf.DNC):
    xx = xxdnc[i]
    yy = yydnc[i]
    print("Divide and Conquer", str(n + 1), "======= [Start]")
    for i in range(conf.KFOLD):
        # Roll the data
        xx = np.roll(xx, int(len(xx) / conf.KFOLD), axis=0)
        yy = np.roll(yy, int(len(yy) / conf.KFOLD), axis=0)

        # Data separation
        xTrain = xx[int(len(xx) / conf.KFOLD):]
        yTrain = yy[int(len(yy) / conf.KFOLD):]
        xEval = xx[:int(len(xx) / conf.KFOLD)]
        yEval = yy[:int(len(yy) / conf.KFOLD)]

        if not conf.USE_NOACTIVITY:
            # Remove NoActivity from ys
            yTrain = yTrain[:, 1:]
            yEval = yEval[:, 1:]

            # If there exists only one action, convert Y to binary form
            if yEval.shape[1] == 1:
                yTrain = ku.to_categorical(yTrain)
                yEval = ku.to_categorical(yEval)

        # Setup Keras Checkpoint
        checkpoint = kc.ModelCheckpoint(
            outputDir + "Checkpoint_DNC" + str(n + 1) + "_K" + str(i + 1) + "_EPOCH{epoch}_ACC{val_acc:.6f}.h5", period=conf.CP_PERIOD)

        # Fit model (learn)
        print(
            str(i + 1) + " th fitting started. Endpoint is " + str(conf.KFOLD) + " th.")

        model.fit(
            xTrain,
            yTrain,
            batch_size=conf.BATCH_SIZE,
            epochs=conf.N_EPOCH,
            verbose=1,
            callbacks=[tensorboard, checkpoint],
            validation_data=(xEval, yEval))  # , validation_freq=2)
    print("Divide and Conquer", str(n + 1), "======= [End]")

Divide and Conquer 1 ======= [Start]
1 th fitting started. Endpoint is 2 th.
Instructions for updating:
Use tf.cast instead.
Train on 60 samples, validate on 60 samples
Epoch 1/512


In [ ]:
# Saving model
print("Saving model & model information...")
modelYML = model.to_yaml()
with open(outputDir + "model.yml", "w") as yml:
    yml.write(modelYML)
modelJSON = model.to_json()
with open(outputDir + "model.json", "w") as json:
    json.write(modelJSON)
model.save(outputDir + "model.h5")
print('Model saved! Congratulations! You finished all processes of ML!')

In [ ]:
# Finished